In [260]:
import pandas as pd

from pandas import read_csv

import csv

class Phylogeny_Data():
    """This class hold information and lists of strain objects
    To represent the strains we are using in our phylogeny"""

    names = []
    
    ##Holds an aligned sequence file
    alignment_file = ""
    
    strains = []
    
    def __init__(self):
        """Creates a Phylogeny_Data obj
        Takes in self"""
    
        alignment_file = ""
        names = []
    
    def get_names(self):
        """Returns a list of names based on the current alignment_file
        Takes in self"""
    
        names = []
    
        file = open(self.alignment_file)
        
        for line in file:

            ##Split up into list of words
            words = line.split()
            
            if len(words) == 0:
                self.names = names
                file.close()
                return names
            else:
                ##take first word
                name = words[0]
                ##add to list
                names.append(name)
                
    def get_strains(self):
        """Returns a list of strain objects from our alignment data
        Takes in self"""
      
        for self.name in self.names:

            genome = Strain()
            genome.name = self.name
            genome.sequences = []
            
            file = open(self.alignment_file)
        
            for line in file:
                ##Split up into list of words
                words = line.split()
            
                if len(words) == 0:
                    continue
                elif words[0] == self.name:
                    genome.sequences.append(words[1])
                elif words[0] == "end":
                    break
                    
            self.strains.append(genome)
    
        file.close()
        
        return self.strains
    
    def compare_seq(self, selected_strain, selected_sequence):
        """function takes in an object strain, and an int sequence (out of 519 sequences)
        compares selected sequence of strain to all other aligned sequences of other strains
        example parameter (Bat_Coronavirus, 342)
        return: a dict holding whats being compared to base as the key and its substitution rate as the value"""
        """example dict
        'Beluga_coronavirus_sequence_342', '3/60 substitutions took place with Bat_coronavirus_sequence_342'
        'Feline_coronavirus_sequence_233', '12/60 substitutions took place with Bat_coronavirus_sequence_233'
        """
        # if counting deletions
        total_comparisons = 60
    
        # if ignoring deletions, counter reset after each comparison
        non_deletion_comparisons = 0
    
        # number of nucleotide matches, reset after each comparison
        matches = 0
    
        # dict to hold compared strain and sequence as key, sub rate with selected strain as value
        substitution_rates = []
    
        # list to hold all aligned sequences of nth alignment to be compared relative to chosen strain
        alignments = []
        
        # String to hold all matches within a comparison ('*' means that the nucleotides did not match)
        matches_as_string = ""
    
        # grabbing selected sequence as bases of comparison
        selected_sequence_as_string = selected_strain.sequences[selected_sequence]

        """adding nth sequence alignment for all 30 strains to list including selected sequence. we will deal with 
        avoiding comparing sequence to itself below"""
        for element in self.strains:
            alignments.append(element.sequences[selected_sequence])
            
        ##print(alignments)

        """iterating thru sequences in list, then iterating thru each sequence and doing comparisons of nucleotides to 
        selected sequence nucleotides, count up matches and nucleotide comparisons, add results to substitution rate 
        dict, not comparing strain sequence to itself"""
    
        # iterating thru 30 selected sequences
        for seq in alignments:
            
            # reset count
            count = 0
            # reset match string
            matches_as_string = ""
            
            #print(seq)
            #print(selected_sequence_as_string)
            
            # do not compare if sequences are of the same strain
            if not seq == selected_sequence_as_string:
                # iterating through 60 char sequence, ignoring deletions, adding up matches
                for nucleotide in seq:
                    
                    ##print(str(nucleotide) + "," + str(selected_sequence_as_string[count]))
                    
                    # check for deletion, add one if no deletion
                    if not nucleotide == "-" and not selected_sequence_as_string[count] == "-":
                        non_deletion_comparisons += 1
                        
                        # check for match, add to count if so
                        if nucleotide == selected_sequence_as_string[count]:
                            matches += 1
                            
                            #print(str(nucleotide) + "," + str(selected_sequence_as_string[count]))
                            
                            ##add to matches string
                            matches_as_string += selected_sequence_as_string[count]
                        else:
                            matches_as_string += "*"
                            
                        # if deletion detected, continue
                    else:
                        ##add to matched string
                        matches_as_string += "-"
                        
                        # add to count
                        count += 1
                        
                        continue
                            
                    # add to count
                    count += 1
                    
                """after each individual sequence comparison is made, add sub rate to dict, and reset num of match and 
                num of comparison counter for new sequence comparison"""
                non_matches = non_deletion_comparisons - matches
                non_matches_as_fraction = str(non_matches) + "/" + str(non_deletion_comparisons)
                
                # testing results
                #print(matches)
                #print(non_deletion_comparisons)
                #print(non_matches)
                #print(non_matches_as_fraction)
                #print(matches_as_string)
            
                ## ensure that we don't divide by zero
                if non_deletion_comparisons == 0:
                    sub_rate = 0.00
                else:
                    sub_rate = non_matches / non_deletion_comparisons
                
                value = sub_rate
                
                data = {"Sequence ID": str(self.strains[alignments.index(seq)].name) 
                            + "_sequence_" + str(selected_sequence + 1), 
                            "Substitution Rate" : value, "Matched Nucleotides" : matches_as_string}
                
                substitution_rates.append(data)
                                          
                matches = 0
                non_deletion_comparisons = 0
            # if sequence is compared against itself, skip and continue
            else:
                continue
        # after all 29 strain sequences are compared to the selected strain sequence, return dict with sub rate info
        return substitution_rates
        
    def create_table(self, sub_rates, strain):
        """Returns nothing, displays a table of the substitution analysis results
        Takes in a dict of seqs and associated sub rates and ase strain name"""
    
        ##explictly name columns
        col_name =["Sequence ID","Substitution Rate", "Matched Nucleotides"]
        
        with open("export.csv", 'w') as csvFile:
            wr = csv.DictWriter(csvFile, fieldnames = col_name, lineterminator = '\n')
            wr.writeheader()
            for element in sub_rates:
                wr.writerow(element)
                
        #Load the text version of the table (a csv file) into python using pandas
        feature_table = pd.read_csv("export.csv",sep="\t")
        
        print(feature_table)
        
        feature_table
              
class Strain():
    """This class hold info about the strain we are analyzing"""

    sequences = []
    ##A list to hold sequences

    name = ""
    ##String to hold strain name

    def __init__(dictionary, seq_name):
    ##Constructs a strain obj using the given parameters
        sequences = dictionary
        name = seq_name
        
    def __init__(self):
    ##No args constructor
        name = ""
        sequences = None
        
    def to_string(self):
    ## returns a string containing obj info
    
        ## name line 
        string = ">" + str(self.name) + "\n"
        
        for self.seq in self.sequences:
            string += str(self.seq + "\n")
            
        return string
    
phylogeny = Phylogeny_Data()

phylogeny.alignment_file = "Alignments.txt"

phylogeny.get_names()

curr_strains = phylogeny.get_strains() 

output = open("Output.txt", 'w')

for strain in curr_strains:
    output.write((strain.to_string()) + "\n")

output.close()

output = open("Output_Comp.txt", 'w')

data = phylogeny.compare_seq(curr_strains[0], 0)

output.write(str(data))

output.close()

phylogeny.create_table(data, curr_strains[0])


    Sequence ID,Substitution Rate,Matched Nucleotides
0   Beluga_Whale_Coronavirus_SW1_sequence_1,0.7777...
1   Bat_Coronavirus_BM48-31_sequence_1,0.636363636...
2   Night-heron_Coronavirus_HKU19_sequence_1,0.592...
3   Wigeon_Coronavirus_HKU20_sequence_1,0.47368421...
4   Common-moorhen_Coronavirus_HKU21_sequence_1,0....
5   Betacoronavirus_HKU24_Strain_HKU_sequence_1,0....
6   Rodent_Coronavirus_Isolate_RtMru_sequence_1,0....
7   Rabbit_Coronavirus_HKU14_sequence_1,0.78260869...
8   Bovine_Coronavirus_Isolate_BCoV-_sequence_1,0....
9   Human_Coronavirus_OC43_Strain_AT_sequence_1,0....
10  Canada_Goose_Coronavirus_Strain__sequence_1,0....
11  Duck_Coronavirus_Isolate_DK/GD/2_sequence_1,0....
12  Turkey_coronavirus_sequence_1,0.625,----------...
13  Betacoronavirus_England_1_sequence_1,0.6296296...
14  Middle_East_Coronavirus_Isolate__sequence_1,0....
15  Thrush_Coronavirus_HKU12-600_sequence_1,0.6296...
16  Magpie-robin_Coronavirus_HKU18_sequence_1,0.51...
17  Munia_Coronavirus_HKU13-